### Modeling Objective

The goal of this notebook is to:
- Train churn prediction models
- Handle class imbalance correctly
- Compare baseline vs tree-based models
- Evaluate models using business-relevant metrics

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve
)

In [2]:
df = pd.read_csv("../data/processed/customer_churn_features.csv")
df.head()

,total_orders,avg_order_value,total_spend,recency_days,customer_tenure_days,orders_last_30d,orders_last_60d,orders_last_90d,avg_discount,discount_usage_rate,avg_kpt,avg_rider_wait,avg_distance,churn
0,3,453.6000,1360.80,122,27,0.0,0.0,0.0,99.000000,1.000000,16.483333,4.133333,5.0,1
1,1,1332.4000,1332.40,21,0,1.0,1.0,1.0,198.050000,1.000000,14.630000,6.100000,3.0,0
2,1,1352.4000,1352.40,49,0,0.0,1.0,1.0,270.000000,1.000000,29.470000,2.900000,2.0,1
3,4,805.6225,3222.49,53,58,0.0,2.0,3.0,103.740000,1.000000,18.772500,2.750000,5.5,1
4,3,673.0500,2019.15,68,48,0.0,0.0,1.0,37.333333,0.333333,15.866667,3.333333,3.0,1


In [8]:
## Feature / Target Split
X = df.drop(columns=["churn"])
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

### Baseline Model: Logistic Regression

In [5]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

log_reg = LogisticRegression(
    class_weight="balanced",
    max_iter=1000,
    random_state=42
)

log_reg.fit(X_train_scaled, y_train)

y_pred_lr = log_reg.predict(X_test_scaled)
y_prob_lr = log_reg.predict_proba(X_test_scaled)[:, 1]

In [6]:
print("ROC-AUC:", roc_auc_score(y_test, y_prob_lr))
print(classification_report(y_test, y_pred_lr))

ROC-AUC: 0.9998810354277742
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       752
           1       1.00      0.99      1.00      2135

    accuracy                           0.99      2887
   macro avg       0.99      1.00      0.99      2887
weighted avg       0.99      0.99      0.99      2887



## Random Forest Model

In [9]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    class_weight="balanced",
    random_state=42
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

In [10]:
print("ROC-AUC:", roc_auc_score(y_test, y_prob_rf))
print(classification_report(y_test, y_pred_rf))

ROC-AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       752
           1       1.00      1.00      1.00      2135

    accuracy                           1.00      2887
   macro avg       1.00      1.00      1.00      2887
weighted avg       1.00      1.00      1.00      2887



### Feature Importance

In [12]:
feature_importance = pd.Series(
    rf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

feature_importance.head(10)

recency_days            0.410877
orders_last_30d         0.335416
orders_last_60d         0.164422
orders_last_90d         0.053838
customer_tenure_days    0.014382
total_spend             0.006194
total_orders            0.005684
avg_discount            0.003482
avg_kpt                 0.002057
discount_usage_rate     0.001353
dtype: float64

### Threshold Tuning

In [13]:
precision, recall, thresholds = precision_recall_curve(y_test, y_prob_rf)

pr_df = pd.DataFrame({
    "threshold": thresholds,
    "precision": precision[:-1],
    "recall": recall[:-1]
})

pr_df.head()

,threshold,precision,recall
0,0.000000,0.739522,1.0
1,0.000031,0.759246,1.0
2,0.000047,0.760599,1.0
3,0.000074,0.761141,1.0
4,0.000094,0.762772,1.0


In [14]:
optimal_threshold = pr_df[pr_df["recall"] >= 0.80]["threshold"].min()
optimal_threshold

np.float64(0.0)

### Confusion Matrix Interpretation

In [16]:
confusion_matrix(y_test, y_pred_rf)

array([[ 752,    0],
       [   0, 2135]])

## XGBoost Model (Gradient Boosting)

XGBoost is used to capture complex non-linear patterns and interactions
between customer behavior, pricing, and operational features.

In [18]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.5 MB/s  0:00:00m 6.8 MB/s eta 0:00:01


In [20]:
from xgboost import XGBClassifier

In [21]:
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
    objective="binary:logistic",
    eval_metric="auc",
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.8
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes f

In [22]:
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]
y_pred_xgb = xgb_model.predict(X_test)

print("ROC-AUC:", roc_auc_score(y_test, y_prob_xgb))
print(classification_report(y_test, y_pred_xgb))

ROC-AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       752
           1       1.00      1.00      1.00      2135

    accuracy                           1.00      2887
   macro avg       1.00      1.00      1.00      2887
weighted avg       1.00      1.00      1.00      2887



In [27]:
confusion_matrix(y_test, y_pred_xgb)

array([[ 752,    0],
       [   0, 2135]])

## Model Comparison Summary

- Logistic Regression provides a strong interpretable baseline
- Random Forest captures non-linear behavior and improves recall
- XGBoost delivers the best trade-off between recall and precision
  after threshold tuning

## Final Model Selection

XGBoost with a business-optimized decision threshold is selected
as the final model for churn prediction due to its superior
performance on recall and ROC-AUC.